## Building a Generative Adversarial Network (GAN)

Generative adevrsarial networks (GANs) are a class of networks that were introduced by Ian Goodfellow in 2014. In GANs, two neural networks play off against one another as adversaries in an actor-critic model, where one is the creator and the other the scrutinizer. The creator, referred to as the generator network, tries to create samples that will fool the scrutinizer, the discriminator network. These two increasingly play off against one another, with the generator network creating increasingly believable samples and the discriminator network getting increasingly good at spotting the samples.

#### Import the Data

In [ ]:
import tensorflow as tf
import numpy as np

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
training_data = (mnist.train.images - 0.5) / 0.5

#### Build the Generator

In [ ]:
def generator(x, initializer):
    
    layer_1 = tf.layers.dense(x, units=256, activation=tf.nn.relu, kernel_initializer=initializer,
                              bias_initializer=initializer, name='input_layer')
    
    layer_2 = tf.layers.dense(layer_1, units=512, activation=tf.nn.relu, kernel_initializer=initializer,
                              bias_initializer=initializer, name='hidden_layer_1')
    
    layer_3 = tf.layers.dense(layer_2, units=1024, activation=tf.nn.relu, kernel_initializer=initializer,
                              bias_initializer=initializer, name='hidden_layer_2')
    
    output_layer = tf.layers.dense(layer_3, units=784, activation=tf.nn.tanh, kernel_initializer=initializer,
                              bias_initializer=initializer, name='generator_output')
    
    return output_layer

#### Build the Discriminator

In [ ]:
def discriminator(x, initializer, dropout_rate):
    
    layer_1 = tf.layers.dense(x, units=1024, activation=tf.nn.relu, kernel_initializer=initializer,
                              bias_initializer=initializer, name='input_layer')
    dropout_1 = tf.layers.dropout(inputs=layer_1, rate=dropout_rate, training=True)

    
    layer_2 = tf.layers.dense(layer_1, units=512, activation=tf.nn.relu, kernel_initializer=initializer,
                              bias_initializer=initializer, name='disc_layer_1')
    dropout_2 = tf.layers.dropout(inputs=layer_2, rate=dropout_rate, training=True)
    
    
    layer_3 = tf.layers.dense(layer_2, units=256, activation=tf.nn.relu, kernel_initializer=initializer,
                              bias_initializer=initializer, name='disc_layer_2')
    dropout_3 = tf.layers.dropout(inputs=layer_3, rate=dropout_rate, training=True)
    
    
    output_layer = tf.layers.dense(layer_3, units=1, activation=tf.sigmoid, kernel_initializer=initializer,
                              bias_initializer=initializer, name='disc_output')
    
    return output_layer

#### Create the training parameters

In [ ]:
learning_rate = 0.0002
batch_size = 100
epochs = 100
dropout_rate=0.5

#### Initialize the input data placeholder for the Generator, z, and the Discriminator, x

In [ ]:
z = tf.placeholder(tf.float32, shape=(None, 100))
x = tf.placeholder(tf.float32, shape=(None, 784))

#### Create the weight and bias initializer

In [ ]:
initializer = tf.contrib.layers.xavier_initializer()

#### Initialize the Generator and Discriminator Functions

In [ ]:
G = generator(z, initializer)

with tf.variable_scope('discriminator_scope') as scope:
    disc_real = discriminator(x, initializer, 0.5)
    scope.reuse_variables()
    disc_fake = discriminator(G, initializer, 0.5)

#### Initialize the Loss and Optimizers

In [ ]:
epsilon = 1e-2
disc_loss = tf.reduce_mean(-tf.log(disc_real + epsilon) - tf.log(1 - disc_fake + epsilon))
gen_loss = tf.reduce_mean(-tf.log(disc_fake + epsilon))

disc_optim = tf.train.AdamOptimizer(lr).minimize(disc_loss)
gen_optim = tf.train.AdamOptimizer(lr).minimize(gen_loss)

#### Run the Training Cycle

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())   
    for epoch in range(epochs):
        
        ## Define the loss to update as a list
        gen_loss = []
        disc_loss = []
        
        ## Run the training iteration
        for iter in range(training_data.shape[0] // batch_size):
            
            ## Batch the input for the discriminator
            x_prime = training_data[iter*batch_size:(iter+1)*batch_size]
            z_prime = np.random.normal(0, 1, (batch_size, 100))

            ## Run the discriminator session
            _, DLoss = sess.run([disc_optim, disc_loss], {x: x_prime, z: z_prime, drop_out: 0.3})
            disc_loss.append(DLoss)

            ## Run the generator session 
            z_prime = np.random.normal(0, 1, (batch_size, 100))
            _, GLoss = sess.run([gen_optim, gen_loss], {z: z_prime, drop_out: 0.3})
            gen_loss.append(GLoss)
            
        if epoch % 5 == 0:
            print('[%d/%d] - loss_d: %.3f, loss_g: %.3f' % ((epoch + 1), epochs, np.mean(D_losses), np.mean(G_losses)))